In [ ]:
## 사전 실행 코드
import polars as pl
df_boxoffice = pl.read_csv("D:/book/polars/data/movie/KOBIS_기간별박스오피스.csv", skip_rows = 4, try_parse_dates=True)
boxoffice_cols = df_boxoffice.columns
boxoffice_cols[4] = '매출액점유율'
df_boxoffice.columns = [i.replace(' ', '') for i in boxoffice_cols]
df_boxoffice = df_boxoffice.with_columns(
    pl.col('매출액', '누적매출액').str.replace_all(',', '').cast(pl.Int64),
    pl.col('관객수', '누적관객수', '스크린수', '상영횟수').str.replace_all(',', '').cast(pl.Int64)    
)
df_boxoffice = df_boxoffice.with_columns(
    pl.col('장르').str.split(','), 
    pl.col('배우').str.split(','), 
)
df_boxoffice = (df_boxoffice.filter(
    pl.col('순위').is_not_null()).
                filter(~(pl.col('장르').list.contains('성인물(에로)'))).
                filter(pl.col('매출액') != 0).
                filter(~((pl.col('관객수') <= 1) & (pl.col('누적관객수') <= 1) & 
                         (pl.col('스크린수') <= 1) & (pl.col('상영횟수') <= 1)))
                )
import altair as alt
import plotly.express as px

# 8장 폴라스로 영화 데이터 생생하게 그리기

## 8.1	폴라스 시각화의 특징 알아보기

## 8.2	폴라스로 손쉽게 데이터 시각화하기

## 8.3	영화 데이터를 시각적으로 풍부하게 표현하기

[8.3.1 결과확인: https://estndard.github.io/polars/8.3.1.html](https://estndard.github.io/polars/8.3.1.html)

### 8.3.1	히스토그램으로 살펴보는 관객 수와 개봉 경향

In [ ]:
df_boxoffice.filter(pl.col('순위') <= 100).select('스크린수').to_series().plot.hist()

In [ ]:
import altair as alt
(df_boxoffice.filter(pl.col('순위') <= 100).select('관객수')
	.plot.bar(x = alt.X('관객수', title = '관객수').bin(maxbins = 30), y = alt.Y('count()', title='영화편수')))

In [ ]:
import plotly.express as px

In [ ]:
fig = px.histogram(  ## plotly.express의 히스토그램 함수
    ## 폴라스 데이터프레임에서 순위 100위 이하 필터링하고, 
    ## X축에 상영횟수 열을 매핑하고 bin의 수를 20개로 설정
	df_boxoffice.filter(pl.col('순위') <= 100), x = '상영횟수', nbins = 20)  
fig.show()

In [ ]:
fig = px.histogram(  ## plotly.express의 히스토그램 함수
	(df_boxoffice.filter(pl.col('순위') <= 100).with_columns(pl.col('개봉일').dt.weekday().alias('개봉요일'))		.sort('개봉요일')
	.with_columns(pl.when(pl.col('개봉요일') == 1).then(pl.lit('월'))
		.when(pl.col('개봉요일') == 2).then(pl.lit('화'))
		.when(pl.col('개봉요일') == 3).then(pl.lit('수'))
		.when(pl.col('개봉요일') == 4).then(pl.lit('목'))
	.when(pl.col('개봉요일') == 5).then(pl.lit('금'))
	.when(pl.col('개봉요일') == 6).then(pl.lit('토'))
	.otherwise(pl.lit('일')).alias('개봉요일'))),
	x = "개봉요일", nbins = 7, text_auto = True)
fig.show()

In [ ]:
fig = px.histogram(  ## plotly.express의 히스토그램 함수
	(df_boxoffice.filter(pl.col('개봉일').dt.year() == 2024, pl.col('순위') <= 100)
	.with_columns(pl.col('개봉일').dt.month().alias('개봉월'))),  ## 순위 100위 이하 필터링
    ## X축에 상영횟수 열을 매핑하고 bin의 수를 20개로 설정
    x = "개봉월", nbins = 12, color = '대표국적', text_auto = True)
fig.show()

### 8.3.2	산점도로 시각화하는 스크린 수와 매출액 관계

[8.3.2 결과확인: https://estndard.github.io/polars/8.3.2.html](https://estndard.github.io/polars/8.3.2.html)

In [ ]:
(df_boxoffice.filter(pl.col('순위') <= 100)
	.plot.point(x = '스크린수', y = '매출액', color = '대표국적'))

In [ ]:
fig = px.scatter(df_boxoffice.filter(pl.col('순위') <= 100)
	.select(pl.col('스크린수'), pl.col('매출액'), pl.col('대표국적')),
	x = '스크린수', y = '매출액', color = '대표국적', trendline = 'lowess')
fig.show()

### 8.3.3	페어 플롯으로 한눈에 비교하는 변수 간 관계

[8.3.3 결과확인: https://estndard.github.io/polars/8.3.3.html](https://estndard.github.io/polars/8.3.3.html)

In [ ]:
(df_boxoffice.filter(pl.col('순위') <= 100)
             
	.select(['매출액', '누적매출액', '관객수', '누적관객수', '스크린수', '상영횟수'])
	
             .plot.point(x = alt.X(alt.repeat("column"), type ='quantitative'), 

		                         y = alt.Y(alt.repeat("row"), type ='quantitative'))
			             .repeat(row = ['매출액', '스크린수', '누적매출액'], column = [ '관객수', '상영횟수', '누적관객수'])) 

In [ ]:
fig = px.scatter_matrix(
    df_boxoffice.select(['매출액', '누적매출액', '관객수', '누적관객수', '스크린수', '상영횟수']).to_pandas())
fig.show()

### 8.3.4	막대그래프로 비교한 국적별 평균 관객 수와 객단가

[8.3.4 결과확인: https://estndard.github.io/polars/8.3.4.html](https://estndard.github.io/polars/8.3.4.html)

In [ ]:
(df_boxoffice.filter(pl.col('순위') <= 100)
	.group_by('대표국적').agg(pl.col('관객수').mean().alias('평균관객수'))
	.plot.bar(x = alt.X('대표국적').sort('-y'), y = '평균관객수').properties(width = 300, height = 300))

In [ ]:
fig = px.bar((df_boxoffice.filter(pl.col('순위') <= 100).group_by('대표국적')
	.agg((pl.col('매출액').sum() / pl.col('관객수').sum()).round(1).alias('객단가')) 
	.sort('객단가', descending = True)), 
	x = '대표국적', y = '객단가', color = '대표국적', text = '객단가',
	color_discrete_map = {"한국": "red", "일본": "gray", "프랑스": "gray", "미국": "gray", "영국": "gray", 
		"호주": "gray", "대만": "gray", "중국": "gray"})
fig.show()

In [ ]:
fig = px.bar(
	(df_boxoffice.filter(pl.col('순위') <= 100, pl.col('대표국적').is_in(["한국", "미국", "일본"]))
		.with_columns(
			pl.when(pl.col('등급') == "15세관람가,15세이상관람가")
			    .then(pl.lit("15세이상관람가")).otherwise(pl.col('등급')).alias('등급'))
		.group_by('등급', '대표국적').agg(pl.col('관객수').mean().round(1).alias('관객수'))), 
	x = '등급', y = '관객수', facet_col = '대표국적', text = '관객수', 
	category_orders = {"대표국적": ["한국", "미국", "일본"], 
		"등급": ["전체관람가", "12세이상관람가", "15세이상관람가", "청소년관람불가"]})
fig.show()

### 8.3.5	선 그래프로 비교하는 〈파묘〉와 〈범죄도시 4〉 관객 추이

[8.3.5 결과확인: https://estndard.github.io/polars/8.3.5.html](https://estndard.github.io/polars/8.3.5.html)

In [ ]:
df_exhuma = (pl.read_csv("./KOBIS_일자별_통계정보_파묘.csv", skip_rows = 3, try_parse_dates = True)
	.with_columns(pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수')
		.str.replace_all(',', '').cast(pl.Int64),
		pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64))
	.with_columns(pl.col('매출액').diff().alias('매출액변화'), pl.col('관객수').diff().alias('관객수변화'))
	.with_columns(pl.lit('파묘').alias('영화명')))

In [ ]:
df_roundup = (pl.read_csv("./KOBIS_일자별_통계정보_범죄도시.csv", skip_rows = 3, try_parse_dates = True)
	.with_columns(pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수')
		.str.replace_all(',', '').cast(pl.Int64),
		pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64))
	.with_columns(pl.col('매출액').diff().alias('매출액변화'), pl.col('관객수').diff().alias('관객수변화'))
	.with_columns(pl.lit('범죄도시4').alias('영화명')))

In [ ]:
df_merge = pl.concat([df_exhuma, df_roundup], how = "vertical")
df_merge

In [ ]:
df_merge.plot.line(x = '날짜', y = '관객수', color = '영화명').properties(width = 800, height = 350)

In [ ]:
fig = px.line(df_merge, x = '날짜', y = '누적관객수', color = '영화명')
fig.show()

### 8.3.6	박스 플롯으로 비교하는 등급 및 장르별 관객 수 분포

[8.3.6 결과확인: https://estndard.github.io/polars/8.3.6.html](https://estndard.github.io/polars/8.3.6.html)

In [ ]:
(df_boxoffice.filter(pl.col('순위') <= 100)	
	.with_columns(pl.when(pl.col('등급') == '15세관람가,15세이상관람가')
		.then(pl.lit('15세이상관람가')).otherwise(pl.col('등급')).alias('등급'))
	.plot.boxplot(x = '대표국적', y = '매출액'))

In [ ]:
fig = px.box((df_boxoffice.filter(pl.col('순위') <= 200).explode('장르')
	.with_columns((pl.col('관객수') / pl.col('상영횟수')).alias('상영횟수별관객수'))), 
	x = '장르', y = '상영횟수별관객수', points = "all", 	
	title = '박스오피스 200위의 장르별 상영횟수당 관객수')
fig.show()

## 8.4	폴라스 표로 영화 데이터 정확하게 시각화하기

In [ ]:
## 생략된 코드
## 인사이드아웃
df_insideout = pl.read_csv("./KOBIS_일자별_통계정보_인사이드아웃2.csv", skip_rows = 3, try_parse_dates=True)
df_insideout_cols = df_insideout.columns
df_insideout.columns = [i.replace(' ', '') for i in df_insideout_cols]
df_insideout = df_insideout.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('인사이드 아웃 2').alias('영화명'))

## 베테랑
df_Executioner = pl.read_csv("./KOBIS_일자별_통계정보_베테랑2.csv", skip_rows = 3, try_parse_dates=True)
df_Executioner_cols = df_Executioner.columns
df_Executioner.columns = [i.replace(' ', '') for i in df_Executioner_cols]
df_Executioner = df_Executioner.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('베테랑2').alias('영화명'))

## 파일럿
df_Pilot = pl.read_csv("./KOBIS_일자별_통계정보_파일럿.csv", skip_rows = 3, try_parse_dates=True)
df_Pilot_cols = df_Pilot.columns
df_Pilot.columns = [i.replace(' ', '') for i in df_Pilot_cols]
df_Pilot = df_Pilot.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('파일럿').alias('영화명'))

## 웡카
df_Wonka = pl.read_csv("./KOBIS_일자별_통계정보_웡카.csv", skip_rows = 3, try_parse_dates=True)
df_Wonka_cols = df_Wonka.columns
df_Wonka.columns = [i.replace(' ', '') for i in df_Wonka_cols]
df_Wonka = df_Wonka.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('웡카').alias('영화명'))

## 모아나2
df_Moana2 = pl.read_csv("./KOBIS_일자별_통계정보_모아나2.csv", skip_rows = 3, try_parse_dates=True)
df_Moana2_cols = df_Moana2.columns
df_Moana2.columns = [i.replace(' ', '') for i in df_Moana2_cols]
df_Moana2 = df_Moana2.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('모아나 2').alias('영화명'))

## 소방관관
df_Firefighter = pl.read_csv("./KOBIS_일자별_통계정보_소방관.csv", skip_rows = 3, try_parse_dates=True)
df_Firefighter_cols = df_Firefighter.columns
df_Firefighter.columns = [i.replace(' ', '') for i in df_Firefighter_cols]
df_Firefighter = df_Firefighter.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('소방관').alias('영화명'))

## 하얼빈
df_Harbin = pl.read_csv("./KOBIS_일자별_통계정보_하얼빈.csv", skip_rows = 3, try_parse_dates=True)
df_Harbin_cols = df_Harbin.columns
df_Harbin.columns = [i.replace(' ', '') for i in df_Harbin_cols]
df_Harbin = df_Harbin.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('하얼빈').alias('영화명'))

## 탈주
df_Escape = pl.read_csv("./KOBIS_일자별_통계정보_탈주.csv", skip_rows = 3, try_parse_dates=True)
df_Escape_cols = df_Escape.columns
df_Escape.columns = [i.replace(' ', '') for i in df_Escape_cols]
df_Escape = df_Escape.with_columns(
    pl.col('스크린수', '상영횟수', '좌석수', '매출액', '관객수', '누적매출액', '누적관객수').str.replace_all(',', '').cast(pl.Int64),
    pl.col('스크린점유율', '상영점유율', '좌석점유율', '좌석판매율').str.replace_all('%', '').cast(pl.Float64), 
).with_columns(
    pl.col('매출액').diff().alias('매출액변화'), 
    pl.col('관객수').diff().alias('관객수변화')
).with_columns(pl.lit('탈주').alias('영화명'))

In [ ]:
df_boxoffice_10 = (pl.concat([
		df_exhuma.sort('날짜').head(30), df_roundup.sort('날짜').head(30), df_insideout.sort('날짜').head(30),
		df_Executioner.sort('날짜').head(30), df_Pilot.sort('날짜').head(30), df_Wonka.sort('날짜').head(30), 
		df_Moana2.sort('날짜').head(30), df_Firefighter.sort('날짜').head(30), df_Harbin.sort('날짜').head(30),
		df_Escape.sort('날짜').head(30)], 
	how = "vertical").group_by('영화명').agg(pl.col('관객수').alias('30일관객수')))
df_boxoffice_10

In [ ]:
df_boxoffice_table = (
	df_boxoffice.with_columns(pl.col('배우').list.head(3), 
		pl.when(pl.col('대표국적') == "한국").then(pl.lit("KOR"))
		.when(pl.col('대표국적') == "미국").then(pl.lit("USA")).alias('flag'),
		(pl.col('매출액') / pl.col('관객수')).round(1).alias('객단가'))
	.select(pl.all().exclude(["국적", "대표국적", "배급사", "제작사",  "누적관객수", "누적매출액", 
		"매출액점유율"])).head(10)
.join(df_boxoffice_10, on = '영화명').sort("순위"))

posterURL = [
    "https://www.kobis.or.kr/common/mast/movie/2024/02/febb838d186b46878ca5467759930333.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/04/82a5c26f7bdd447093270311a2caa153.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/06/041e1f5d30ad452ea0c31714aadb8734.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/08/886a61eb398d4cedb346d9ffbf3345c3.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/07/a901202adef143bd8cd3a54fd90e3fa8.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/01/909ae4c66c9a439f98f90afe60ec42fb.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/11/b0a0a5ae940443049c18e22e4787b08b.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/12/090f80c0e45d46569852d1e04418f4ed.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2025/01/a3c89c36722e41068cf0c01308ebef3a.jpg",
    "https://www.kobis.or.kr/common/mast/movie/2024/07/e7d6eec007b84ed381a4f54d7b33147a.jpg"]

df_boxoffice_table = df_boxoffice_table.with_columns(pl.Series("posterURL", posterURL))
df_boxoffice_table

### 8.4.2	GT 객체 생성 방식 비교

In [ ]:
type(df_boxoffice_table.head(10).style)

In [ ]:
from great_tables import GT, md, html, loc, style
gt_boxoffice = GT(df_boxoffice_table.head(10), locale = "ko")
gt_boxoffice

### 8.4.3	표 시각화를 위한 구조 설정

In [ ]:
style_boxoffice = (df_boxoffice_table.head(10)
	.style.tab_header(title = '2024년 박스오피스 Top 10',
		subtitle = md('2024년 상영 영화 중 1,000만 관객이 든 영화는 <strong>파묘</strong>, <strong>범죄도시 4</strong>였고, 미국영화 중 가장 흥행작은 **인사이드아웃 2**였습니다.')))
style_boxoffice

In [ ]:
gt_boxoffice = gt_boxoffice.tab_header(title = '2024년 박스오피스 Top 10',
	subtitle = md('2024년 상영 영화 중 1,000만 관객이 든 영화는 <strong>파묘</strong>, <strong>범죄도시 4</strong>였고, 미국영화 중 가장 흥행작은 **인사이드아웃 2** 였습니다.'))
gt_boxoffice

In [ ]:
style_boxoffice1 = (
	style_boxoffice.tab_stub(rowname_col = '순위', groupname_col = '등급').tab_stubhead('순위'))
style_boxoffice1

In [ ]:
style_boxoffice2 = (style_boxoffice1
	.cols_label(배우 = '대표배우').cols_label(flag = '국가')  ## 배우 열 이름은 대표배우로, flag 열 이름은 국가로 변경
	.cols_move(['감독', '배우', '장르'], '영화명')  ## 영화명 뒤로 감독, 배우, 장르 열 이동 
	.cols_move('flag', '상영횟수')  ## 상영횟수 뒤로 flag 열 이동
.cols_move_to_start('posterURL')  ## 맨 앞으로 posterURL 열 이동
.cols_align(align = "center")  ## 열의 정렬을 중간 정렬로 설정
.cols_width(cases = {"영화명": "80px", "감독": "75px"})  ## 영화명과 감독 열의 너비 지정
.tab_spanner('영화정보', ['감독', '배우', '장르', '개봉일'])  ## 감독, 배우, 장르, 개봉일은 영화정보 스패너로 설정
	.tab_spanner('매출정보', ['매출액', '객단가'])  ## 매출액, 객단가 열은 매출정보 스패너로 설정
	.tab_spanner('관객정보', ['관객수'])  ## 관객수는 관객정보 스패너로 설정
	.tab_spanner('상영정보', ['스크린수', '상영횟수']))  ## 스크린수, 상영횟수는 상영정보 스패너로 설정
style_boxoffice2

### 8.4.4	GT 셀 포매팅과 시각적 조정

In [ ]:
style_boxoffice3 = (style_boxoffice2
	.fmt_currency(columns = ['매출액', '객단가'], currency = "KRW")
	.fmt_integer(columns = ['관객수', '스크린수', '상영횟수'])
	.fmt_date(columns = '개봉일', date_style = "wday_month_day_year", locale = "ko")) 
style_boxoffice3

In [ ]:
style_boxoffice4 = (style_boxoffice3.fmt_flag(columns = "flag", height = "3em")
	.cols_width(cases = {'posterURL': "80px"})
	.fmt_image(columns = 'posterURL', width = "100%")
	.cols_label(posterURL = ""))
style_boxoffice4

In [ ]:
style_boxoffice5 = style_boxoffice4.fmt_nanoplot(columns = "30일관객수", reference_line = "mean")
style_boxoffice5

### 8.4.5	조건부 서식과 강조를 위한 셀 스타일링

In [ ]:
style_boxoffice6 = (style_boxoffice5
	.tab_style(style = style.text(weight = "bold"), locations = loc.body(columns = '영화명'))
		.tab_style(style = style.text(color = "blue"), locations = loc.body(columns='영화명', rows = [0, 1]))
		.tab_style(style = style.text(weight = "bold", size = "xx-large"), locations = loc.title()))
style_boxoffice6

In [ ]:
style_boxoffice7 = (style_boxoffice6
    .tab_style(
        style = style.text(color = pl.when(pl.col('관객수') > 5_000_000).then(pl.lit("red")).otherwise(pl.lit("lightgray"))),	
	 locations = loc.body(columns = '관객수'))
	.tab_style(
		style = style.text(weight = "bold"), 
		locations = loc.body(rows = pl.col('관객수') == pl.col('관객수').max()))	
	.tab_style(
		style = style.fill(color = pl.when(pl.col('객단가') > pl.col('객단가').mean()).then(pl.lit("yellow")).otherwise(pl.lit("lightgray"))),
		locations = loc.body(columns = '객단가'))
	.tab_style(
		style = style.borders(color = "blue", weight = "3px"),
		locations = loc.body(columns = '매출액', rows = pl.col('매출액') > pl.col('매출액').mean())))
style_boxoffice7

In [ ]:
style_boxoffice8 = style_boxoffice7.data_color(
	columns = '스크린수', palette = "Greens", domain = [1000, 3000])
style_boxoffice8

In [ ]:
style_boxoffice9 = (style_boxoffice8
	.tab_source_note(source_note = html('출처: <img src = "https://www.kobis.or.kr/kobis/web/comm/images/comm/logo_comm.png"> (https://www.kobis.or.kr/)'))
	.tab_style(style = style.text(align = "right", weight = "bold"), locations = loc.source_notes()))
style_boxoffice9